In [1]:
# import numpy as np

In [2]:
# import numpy as np

# results = np.load("../results/cifar100/mahalanobis/tiny_imagenet/cifar100_tiny_imagenet_mahalanobis.npz")

In [3]:
# print(results.keys())

# print(results["ind_test"].shape)
# print(results["ind_calib"].shape)
# print(results["ood"].shape)

In [4]:
import pandas as pd
import numpy as np

In [5]:
ind_dataset = "cifar10"
ood_dataset = "svhn"
n = 2



from pathlib import Path
import pandas as pd

path = Path(f"../results/{ind_dataset}_{ood_dataset}_{n}_combo_results.csv")

df = pd.read_csv(path)

# infer how many columns the header has (K)
# with path.open("r", encoding="utf-8") as f:
#     K = len(f.readline().rstrip("\n").split(","))

# df = pd.read_csv(
#     path,
#     engine="python",            # tolerant parser
#     usecols=list(range(K)),     # drop any extra columns in long rows
#     on_bad_lines="skip",        # skip rows that are too short/broken
# )

In [6]:
df

,ind_dataset,ood_dataset,group_id,risk_brierscore_bayesrisk_inner_inner_T_1.0,risk_logscore_excessrisk_central_outer_T_1.0,multidimensional_score,risk_zeroonescore_bayesrisk_inner_inner_T_1.0,risk_zeroonescore_totalrisk_outer_inner_T_1.0,risk_logscore_totalrisk_inner_central_T_1.0,risk_zeroonescore_totalrisk_outer_outer_T_1.0,...,risk_logscore_excessrisk_central_central_T_1.0,risk_sphericalscore_excessrisk_outer_central_T_1.0,risk_brierscore_totalrisk_central_inner_T_1.0,risk_zeroonescore_totalrisk_central_inner_T_1.0,risk_brierscore_excessrisk_inner_central_T_1.0,risk_sphericalscore_excessrisk_central_inner_T_1.0,risk_zeroonescore_totalrisk_inner_outer_T_1.0,risk_logscore_excessrisk_outer_central_T_1.0,risk_zeroonescore_excessrisk_inner_outer_T_1.0,risk_zeroonescore_bayesrisk_outer_inner_T_1.0
0,cifar10,svhn,0,0.958093,0.950890,0.956757,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cifar10,svhn,1,0.962692,0.956492,0.961754,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cifar10,svhn,2,0.954705,0.934243,0.954832,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cifar10,svhn,3,0.955253,0.938749,0.953622,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cifar10,svhn,0,NaN,NaN,0.955192,0.955192,0.955192,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,cifar10,svhn,3,NaN,NaN,0.958324,NaN,NaN,0.959108,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,cifar10,svhn,0,NaN,NaN,0.958827,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.954673
1197,cifar10,svhn,1,NaN,NaN,0.960821,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.955153
1198,cifar10,svhn,2,NaN,NaN,0.955036,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.957677


In [7]:
fixed_cols = ['ind_dataset', 'ood_dataset', 'group_id', 'multidimensional_score']
measure_cols = [col for col in df.columns if col not in fixed_cols]

def get_combination_key(row):
    """Create a key identifying which measures were used (non-NaN)"""
    non_nan_measures = [col for col in measure_cols if pd.notna(row[col])]
    return tuple(sorted(non_nan_measures))

# Add combination key to dataframe
df['combination_key'] = df.apply(get_combination_key, axis=1)

# Calculate statistics for each combination
results_stats = []

for combo_key in df['combination_key'].unique():
    combo_df = df[df['combination_key'] == combo_key]
    
    # Get the measure names for this combination
    measure_names = list(combo_key)
    
    # Calculate mean and std for multidimensional_score (always present)
    multi_mean = combo_df['multidimensional_score'].mean()
    multi_std = combo_df['multidimensional_score'].std()
    
    # Calculate mean and std for each individual measure in this combination
    individual_stats = {}
    for measure in measure_names:
        individual_stats[f'{measure}_mean'] = combo_df[measure].mean()
        individual_stats[f'{measure}_std'] = combo_df[measure].std()
    
    # Create result row
    result_row = {
        'combination': ', '.join(measure_names),
        'n_measures': len(measure_names),
        'n_samples': len(combo_df),
        'multidimensional_score_mean': multi_mean,
        'multidimensional_score_std': multi_std,
        **individual_stats
    }
    
    results_stats.append(result_row)

# Convert to DataFrame and display
stats_df = pd.DataFrame(results_stats)
stats_df = stats_df.sort_values(['n_measures', 'multidimensional_score_mean'], ascending=[True, False])
print(f"Summary statistics for {len(stats_df)} different measure combinations:")
stats_df

Summary statistics for 300 different measure combinations:


,combination,n_measures,n_samples,multidimensional_score_mean,multidimensional_score_std,risk_brierscore_bayesrisk_inner_inner_T_1.0_mean,risk_brierscore_bayesrisk_inner_inner_T_1.0_std,risk_logscore_excessrisk_central_outer_T_1.0_mean,risk_logscore_excessrisk_central_outer_T_1.0_std,risk_zeroonescore_bayesrisk_inner_inner_T_1.0_mean,...,risk_sphericalscore_excessrisk_central_inner_T_1.0_mean,risk_sphericalscore_excessrisk_central_inner_T_1.0_std,risk_zeroonescore_totalrisk_inner_outer_T_1.0_mean,risk_zeroonescore_totalrisk_inner_outer_T_1.0_std,risk_logscore_excessrisk_outer_central_T_1.0_mean,risk_logscore_excessrisk_outer_central_T_1.0_std,risk_zeroonescore_excessrisk_inner_outer_T_1.0_mean,risk_zeroonescore_excessrisk_inner_outer_T_1.0_std,risk_zeroonescore_bayesrisk_outer_inner_T_1.0_mean,risk_zeroonescore_bayesrisk_outer_inner_T_1.0_std
294,"risk_logscore_bayesrisk_outer_outer_T_1.0, ris...",2,4,0.963710,0.001939,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,"risk_logscore_bayesrisk_outer_outer_T_1.0, ris...",2,4,0.963627,0.002421,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,"risk_logscore_bayesrisk_inner_outer_T_1.0, ris...",2,4,0.963622,0.002046,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,"risk_logscore_bayesrisk_central_outer_T_1.0, r...",2,4,0.963471,0.001469,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,"risk_logscore_bayesrisk_inner_outer_T_1.0, ris...",2,4,0.963465,0.002235,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,"risk_brierscore_excessrisk_inner_inner_T_1.0, ...",2,4,0.931379,0.011891,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,"risk_logscore_excessrisk_inner_central_T_1.0, ...",2,4,0.929900,0.014861,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,risk_sphericalscore_excessrisk_central_central...,2,4,0.824603,0.038295,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,risk_sphericalscore_excessrisk_inner_inner_T_1...,2,4,0.824603,0.038295,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Filter rows where multidimensional_score_mean is better than individual measures
def is_multidim_better(row):
    multidim_mean = row['multidimensional_score_mean']
    if pd.isna(multidim_mean):
        return False
    
    # Get all individual measure mean columns for this row
    individual_means = []
    for col in row.index:
        if col.endswith('_mean') and col != 'multidimensional_score_mean':
            if pd.notna(row[col]):
                individual_means.append(row[col])
    
    # Check if multidimensional score is better than all individual measures
    if individual_means:
        return multidim_mean > max(individual_means)
    return False

# Filter and sort the dataframe
filtered_stats_df = stats_df[stats_df.apply(is_multidim_better, axis=1)]
filtered_stats_df = filtered_stats_df.sort_values('multidimensional_score_mean', ascending=False)

print(f"Combinations where multidimensional score outperforms all individual measures ({len(filtered_stats_df)} out of {len(stats_df)}):")
filtered_stats_df


Combinations where multidimensional score outperforms all individual measures (66 out of 300):


,combination,n_measures,n_samples,multidimensional_score_mean,multidimensional_score_std,risk_brierscore_bayesrisk_inner_inner_T_1.0_mean,risk_brierscore_bayesrisk_inner_inner_T_1.0_std,risk_logscore_excessrisk_central_outer_T_1.0_mean,risk_logscore_excessrisk_central_outer_T_1.0_std,risk_zeroonescore_bayesrisk_inner_inner_T_1.0_mean,...,risk_sphericalscore_excessrisk_central_inner_T_1.0_mean,risk_sphericalscore_excessrisk_central_inner_T_1.0_std,risk_zeroonescore_totalrisk_inner_outer_T_1.0_mean,risk_zeroonescore_totalrisk_inner_outer_T_1.0_std,risk_logscore_excessrisk_outer_central_T_1.0_mean,risk_logscore_excessrisk_outer_central_T_1.0_std,risk_zeroonescore_excessrisk_inner_outer_T_1.0_mean,risk_zeroonescore_excessrisk_inner_outer_T_1.0_std,risk_zeroonescore_bayesrisk_outer_inner_T_1.0_mean,risk_zeroonescore_bayesrisk_outer_inner_T_1.0_std
294,"risk_logscore_bayesrisk_outer_outer_T_1.0, ris...",2,4,0.963710,0.001939,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,"risk_logscore_bayesrisk_outer_outer_T_1.0, ris...",2,4,0.963627,0.002421,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,"risk_logscore_bayesrisk_inner_outer_T_1.0, ris...",2,4,0.963622,0.002046,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,"risk_logscore_bayesrisk_central_outer_T_1.0, r...",2,4,0.963471,0.001469,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,"risk_logscore_bayesrisk_inner_outer_T_1.0, ris...",2,4,0.963465,0.002235,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,"risk_brierscore_excessrisk_outer_outer_T_1.0, ...",2,4,0.941616,0.009139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,"risk_brierscore_excessrisk_outer_inner_T_1.0, ...",2,4,0.940174,0.010347,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,risk_sphericalscore_excessrisk_inner_central_T...,2,4,0.934025,0.010611,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,risk_sphericalscore_excessrisk_central_inner_T...,2,4,0.931748,0.011076,NaN,NaN,NaN,NaN,NaN,...,0.929676,0.012439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Add a column showing percentage benefit of multidimensional score over best individual measure
def calculate_benefit_percentage(row):
    multidim_mean = row['multidimensional_score_mean']
    if pd.isna(multidim_mean):
        return None
    
    # Get all individual measure mean columns for this row
    individual_means = []
    for col in row.index:
        if col.endswith('_mean') and col != 'multidimensional_score_mean':
            if pd.notna(row[col]):
                individual_means.append(row[col])
    
    if individual_means:
        max_individual = max(individual_means)
        benefit_percentage = ((multidim_mean - max_individual) / max_individual) * 100
        return benefit_percentage
    return None

filtered_stats_df = filtered_stats_df.copy()
filtered_stats_df['benefit_percentage'] = filtered_stats_df.apply(calculate_benefit_percentage, axis=1)

filtered_stats_df

,combination,n_measures,n_samples,multidimensional_score_mean,multidimensional_score_std,risk_brierscore_bayesrisk_inner_inner_T_1.0_mean,risk_brierscore_bayesrisk_inner_inner_T_1.0_std,risk_logscore_excessrisk_central_outer_T_1.0_mean,risk_logscore_excessrisk_central_outer_T_1.0_std,risk_zeroonescore_bayesrisk_inner_inner_T_1.0_mean,...,risk_sphericalscore_excessrisk_central_inner_T_1.0_std,risk_zeroonescore_totalrisk_inner_outer_T_1.0_mean,risk_zeroonescore_totalrisk_inner_outer_T_1.0_std,risk_logscore_excessrisk_outer_central_T_1.0_mean,risk_logscore_excessrisk_outer_central_T_1.0_std,risk_zeroonescore_excessrisk_inner_outer_T_1.0_mean,risk_zeroonescore_excessrisk_inner_outer_T_1.0_std,risk_zeroonescore_bayesrisk_outer_inner_T_1.0_mean,risk_zeroonescore_bayesrisk_outer_inner_T_1.0_std,benefit_percentage
294,"risk_logscore_bayesrisk_outer_outer_T_1.0, ris...",2,4,0.963710,0.001939,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.074546
297,"risk_logscore_bayesrisk_outer_outer_T_1.0, ris...",2,4,0.963627,0.002421,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065957
182,"risk_logscore_bayesrisk_inner_outer_T_1.0, ris...",2,4,0.963622,0.002046,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049073
271,"risk_logscore_bayesrisk_central_outer_T_1.0, r...",2,4,0.963471,0.001469,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049736
275,"risk_logscore_bayesrisk_inner_outer_T_1.0, ris...",2,4,0.963465,0.002235,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,"risk_brierscore_excessrisk_outer_outer_T_1.0, ...",2,4,0.941616,0.009139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.153507
34,"risk_brierscore_excessrisk_outer_inner_T_1.0, ...",2,4,0.940174,0.010347,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000072
284,risk_sphericalscore_excessrisk_inner_central_T...,2,4,0.934025,0.010611,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.284063
272,risk_sphericalscore_excessrisk_central_inner_T...,2,4,0.931748,0.011076,NaN,NaN,NaN,NaN,NaN,...,0.012439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222854


In [10]:
filtered_stats_df['benefit_percentage'].sort_values(ascending=False)

193    1.472622
72     0.613234
288    0.523176
88     0.408944
163    0.370909
         ...   
206    0.000159
34     0.000072
43     0.000070
195    0.000022
276    0.000006
Name: benefit_percentage, Length: 66, dtype: float64

In [11]:
filtered_stats_df.loc[193].dropna()

combination                                            risk_logscore_excessrisk_inner_central_T_1.0, ...
n_measures                                                                                             2
n_samples                                                                                              4
multidimensional_score_mean                                                                       0.9299
multidimensional_score_std                                                                      0.014861
risk_logscore_excessrisk_inner_central_T_1.0_mean                                               0.916405
risk_logscore_excessrisk_inner_central_T_1.0_std                                                0.014967
risk_zeroonescore_excessrisk_outer_outer_T_1.0_mean                                              0.82336
risk_zeroonescore_excessrisk_outer_outer_T_1.0_std                                              0.039171
benefit_percentage                                     